In [251]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer

import unicodedata as ud
from sklearn.metrics import precision_recall_fscore_support as prfs
import csv
from itertools import izip
import HTMLParser
from sklearn.metrics import classification_report
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import linear_model
from sklearn import linear_model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer as porterStemmer

In [252]:
data = pd.read_excel("C:\\Users\\swapnil sagar\\Documents\\UIC\\Data mining\\project 2\\romneyTweets_Filtered.xlsx");
data.head(9)

,date,time,Anootated tweet,Class
0,NaN,NaN,"1: positive, -1: negative, 0: neutral, 2: mixed",NaN
1,2012-10-16 00:00:00,09:38:08-05:00,Insidious!<e>Mitt Romney</e>'s Bain Helped Phi...,-1.0
2,2012-10-16 00:00:00,10:14:18-05:00,.@WardBrenda @shortwave8669 @allanbourdius you...,-1.0
3,2012-10-16 00:00:00,09:27:16-05:00,<e>Mitt Romney</e> still doesn't <a>believe</a...,-1.0
4,2012-10-16 00:00:00,10:11:43-05:00,<e>Romney</e>'s <a>tax plan</a> deserves a 2nd...,-1.0
5,2012-10-16 00:00:00,10:13:17-05:00,Hope <e>Romney</e> debate prepped w/ the same ...,1.0
6,2012-10-16 00:00:00,10:17:28-05:00,Want to know how <e>Mitt Romney</e> is going t...,-1.0
7,2012-10-16 00:00:00,09:35:55-05:00,If <e>Romney</e> wins the <a>presidential elec...,-1.0
8,2012-10-16 00:00:00,10:28:50-05:00,<e>Romney</e>'s <a>12 million jobs scam </a>re...,-1.0


In [253]:
annotatedTweet = data['Anootated tweet']
actualClass = data['Class']
lengthObama = len(data);

annotatedTweet = annotatedTweet.drop(0)
actualClass = actualClass.drop(0)


annotatedTweetList = annotatedTweet.tolist();
actualClassList = actualClass.tolist();

# For test data

In [235]:
data = pd.read_excel("C:\\Users\\swapnil sagar\\Documents\\UIC\\Data mining\\project 2\\final-testData-no-label-Obama-Romney-tweets.xlsx", sheetname="Romney");
data

,Anootated tweets
1,<e>Romney</e> got 3 less minutes and had to de...
2,<e>Mitt </e>is beating him UP! on his record...
3,I actually like <e>Romney </e>'s response to ...
4,Just for that <a>immigration statement </a>tha...
5,This man <e>Romney </e>is tearing this dude ...
6,"<e>Romney </e>had less to prove, given the la..."
7,"<e>Romney </e>- he comes off as a dumbass, but..."
8,Great <a>closing</a> by <e>Romney </e>. Summe...
9,Just ready for <e>Romney </e> to become presi...
10,"Get him on his trampling on the Constitution, ..."


In [236]:
data = pd.read_excel("C:\\Users\\swapnil sagar\\Documents\\UIC\\Data mining\\project 2\\final-testData-no-label-Obama-Romney-tweets.xlsx", sheetname="Romney");
annotatedTweet = data['Anootated tweets']
#annotatedTweet = annotatedTweet.drop(0)Anootated tweets
annotatedTweetList = annotatedTweet.tolist();

In [248]:
data

,date,time,Anootated tweet,Class
0,NaN,NaN,"1: positive, -1: negative, 0: neutral, 2: mixed",NaN
1,2012-10-16 00:00:00,09:38:08-05:00,Insidious!<e>Mitt Romney</e>'s Bain Helped Phi...,-1.0
2,2012-10-16 00:00:00,10:14:18-05:00,.@WardBrenda @shortwave8669 @allanbourdius you...,-1.0
3,2012-10-16 00:00:00,09:27:16-05:00,<e>Mitt Romney</e> still doesn't <a>believe</a...,-1.0
4,2012-10-16 00:00:00,10:11:43-05:00,<e>Romney</e>'s <a>tax plan</a> deserves a 2nd...,-1.0
5,2012-10-16 00:00:00,10:13:17-05:00,Hope <e>Romney</e> debate prepped w/ the same ...,1.0
6,2012-10-16 00:00:00,10:17:28-05:00,Want to know how <e>Mitt Romney</e> is going t...,-1.0
7,2012-10-16 00:00:00,09:35:55-05:00,If <e>Romney</e> wins the <a>presidential elec...,-1.0
8,2012-10-16 00:00:00,10:28:50-05:00,<e>Romney</e>'s <a>12 million jobs scam </a>re...,-1.0
9,2012-10-16 00:00:00,10:21:31-05:00,<e>Mitt #Romney</e> <a>said</a> that <a> catch...,-1.0


In [254]:
count = 0;

finalFilteredTweet = []

def removeStopWords(tweet):
    filtered_tweet = [];
    
    stop_words = set(stopwords.words('english'));
    word_tokens = word_tokenize(tweet);
    
    for w in word_tokens:
        if w not in stop_words:
            filtered_tweet.append(w);
    
    eachTweet = " ".join(filtered_tweet)  
    return eachTweet
    
    
def preProcessTweets(annotatedTweet):
        
        #remove links
        processedTweet = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', annotatedTweet);
        #remove references with @<word>
        processedTweet = re.sub(r'(\s)@\w+', r'', processedTweet)
        processedTweet = re.sub(r'@\w+', r'', processedTweet)
        
        # remove HTML tags from tweet
        processedTweet = re.sub('<[^<]+?>', '', processedTweet)
        
        #remove exclamation marks - hashtags should be removed?
        processedTweet = re.sub(r'[<>!#@$:,%\?-]+', r' ', processedTweet)
        processedTweet = re.sub(r'[.]+', r'', processedTweet)
        
       # processedTweet = re.sub(r'(\s)#\w+', r'', processedTweet)
       # processedTweet = re.sub(r'#\w+', r'', processedTweet)
        
        # remove extra white spaces
        processedTweet = re.sub(r'\s+', r' ', processedTweet)
        
        #remove "" 
        processedTweet = processedTweet.replace('"', '');
        
        processedTweet = ''.join([i if ord(i) < 128 else ' ' for i in processedTweet])

        #stemming
        stemmer = porterStemmer()
        stemmedTweet = [stemmer.stem(word) for word in processedTweet.split(" ")]
        stemmedTweet = " ".join(stemmedTweet)
        processedTweet = str(stemmedTweet);
        
        processedTweet = processedTweet.replace("'", "");

        #remove numbers from data
        #join
        #processedTweet = " ".join(re.findall('[A-Z][^A-Z]*', processedTweet));

        #processedTweet = removeStopWords(processedTweet); --- Remove stop words in the end
        
        return processedTweet;

    
for every_tweet in annotatedTweetList:
    
    count = count +1     
    tweet = preProcessTweets(every_tweet).encode('ascii', 'ignore').strip();
    #print type(tweet)

    finalFilteredTweet.append(tweet);        

In [255]:
len(finalFilteredTweet)

5648

In [256]:
finalFilteredTweet[:5]

['insidi mitt romney bain help philip morri get US high schooler hook On cigarett via',
 'you mean like romney cheat in primari',
 'mitt romney still doesnt believ that we have a black presid',
 'romney tax plan deserv a 2nd look becaus he ha a secret one that differ than the one he been lie about',
 'hope romney debat prep w/ the same peopl as last time']

In [198]:
trainData, testTrainData , y_train, y_test = train_test_split(finalFilteredTweet, actualClassList, test_size = 0.15, random_state = 40)

In [240]:
len(trainData)
len(testTrainData)

848

In [257]:
vectorizer = TfidfVectorizer(min_df=0.00125,
                             max_df = 0.70,
                             sublinear_tf=True,
                             use_idf=True,
                            stop_words=u'english',
                            analyzer='word',
                            ngram_range=(1,6),lowercase=True)

In [258]:
#train_vectors = vectorizer.fit_transform(trainData);
#test_vectors = vectorizer.transform(testTrainData);
total_vectors = vectorizer.fit_transform(finalFilteredTweet);

#knn = KNeighborsClassifier()
Classifier_sgd = linear_model.SGDClassifier()

In [242]:
total_vectors_test = vectorizer.transform(finalFilteredTweet);

NotFittedError: TfidfVectorizer - Vocabulary wasn't fitted.

In [259]:
#classifier_sgd.fit(train_vectors, y_train)
classifier_sgd.fit(total_vectors, actualClassList)
#knn.fit(train_vectors, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [228]:
predicted_labels = classifier_sgd.predict(total_vectors_test)
#predicted_labels = knn.predict(test_vectors)
predicted_labelsList = predicted_labels.tolist();
#classifier_sgd.score(test_vectors, y_test)

In [32]:
target_classifiers = ['-1', '0', '1']
print(classification_report(y_test, predicted_labelsList, target_names=target_classifiers))

             precision    recall  f1-score   support

         -1       0.61      0.80      0.69       425
          0       0.45      0.31      0.37       252
          1       0.55      0.40      0.46       171

avg / total       0.55      0.57      0.55       848



# Get Predicted list for test Data

In [70]:
submission = pd.DataFrame(columns=['id',';;','Class'])
submission

,id,;;,Class


In [229]:
i = 0
for num in predicted_labelsList:
    submission.loc[i+1] = [i+1,';;',predicted_labelsList[i]]

    i=i+1

In [230]:
np.savetxt(r'C:\Users\swapnil sagar\Documents\UIC\Data mining\project 2\Swapnil_Sagar_Khusbhu_Durge_Romney.txt', submission.values, fmt='%d %s %d')
submission.head(7)

,id,;;,Class
1,1,;;,-1.0
2,2,;;,-1.0
3,3,;;,-1.0
4,4,;;,-1.0
5,5,;;,-1.0
6,6,;;,1.0
7,7,;;,0.0


In [159]:
cvScores = cross_val_score(classifier_sgd, total_vectors, actualClassList, cv=10,scoring='f1_weighted')
print (cvScores)

[ 0.55676973  0.49721168  0.52928048  0.52719085  0.53729913  0.56985312
  0.58376228  0.48624701  0.54532505  0.58838464]


In [160]:
print ("Mean: ", cvScores.mean())
print ("Minimum: ", cvScores.min())
print ("Maximum: ", cvScores.max())

('Mean: ', 0.54213239781325251)
('Minimum: ', 0.48624700967896156)
('Maximum: ', 0.58838463715114719)


In [161]:
cvPredict = cross_val_predict(classifier_sgd, total_vectors, actualClassList, cv=10)
print (cvPredict)

[-1. -1. -1. ..., -1. -1. -1.]


In [162]:
print (classification_report(actualClassList,cvPredict))

             precision    recall  f1-score   support

       -1.0       0.63      0.76      0.69      2893
        0.0       0.42      0.32      0.36      1680
        1.0       0.51      0.40      0.45      1075

avg / total       0.54      0.56      0.55      5648



In [39]:
bdt = AdaBoostClassifier(classifier_sgd,
                         algorithm="SAMME",
                         n_estimators=200)

In [40]:
cvPredict = cross_val_predict(bdt, total_vectors, actualClassList, cv=10)
print (cvPredict)

[-1. -1. -1. ...,  0. -1.  0.]


In [41]:
print (classification_report(actualClassList,cvPredict))

             precision    recall  f1-score   support

       -1.0       0.62      0.63      0.63      2893
        0.0       0.36      0.37      0.37      1680
        1.0       0.41      0.39      0.40      1075

avg / total       0.50      0.51      0.50      5648



In [26]:
clf1 = LogisticRegression(random_state=1)

In [28]:
cvPredict = cross_val_predict(clf1, total_vectors, actualClassList, cv=10)
print (cvPredict)

[-1. -1. -1. ..., -1. -1. -1.]


In [29]:
print (classification_report(actualClassList,cvPredict))

             precision    recall  f1-score   support

       -1.0       0.60      0.84      0.70      2893
        0.0       0.43      0.27      0.33      1680
        1.0       0.58      0.31      0.40      1075

avg / total       0.55      0.57      0.53      5648

